In [4]:
import time
import json
import hmac
import requests
import hashlib
base_url = "https://fapi.pi42.com/"

api_key = "b082b62c54596daa2bb71ff276894a12"
api_secret = "4c3d89a7bfb1f0223a72ee7bcad34202"

def generate_signature(api_secret, data_to_sign):
    return hmac.new(api_secret.encode('utf-8'), data_to_sign.encode('utf-8'), hashlib.sha256).hexdigest()

def place_order(symbol, limit_price, order_type, quantity, side, reduce_only):

    
    timestamp = str(int(time.time() * 1000))

    print("API KEY: ", api_key)
    print("API SECRET: ", api_secret)

    params = {
        'timestamp': timestamp,       
        'placeType': 'ORDER_FORM',   
        'quantity': quantity,            
        'side': side,                
        'symbol': symbol,          
        'type': order_type,             
        'reduceOnly': reduce_only,          
        'marginAsset': 'INR',          
        'deviceType': 'WEB',          
        'userCategory': 'EXTERNAL',    
        'price': limit_price,         
        'stopPrice': limit_price,
    }

    data_to_sign = json.dumps(params, separators=(',', ':'))
    signature = generate_signature(api_secret, data_to_sign)

    headers = {
        'api-key': api_key,
        'signature': signature,
    }

    try:
        print(f'{base_url}v1/order/place-order')
        print("Payload being sent:", json.dumps(params, indent=4))
        print("Headers being sent:", headers)
        response = requests.post(f'{base_url}v1/order/place-order', json=params, headers=headers)

        response.raise_for_status()

        response_data = response.json()
        return response_data

    except requests.exceptions.HTTPError as err:
        print(f"Error: {err.response.text if err.response else err}")

    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

def get_24hr_ticker_update(contract_pair):

    # Validate the input
    if not contract_pair:
        print("Invalid contract pair. Please enter a valid contract pair (e.g., btc, eth).")
        return

    # Construct the full URL for the API request using the provided contract pair
    full_url = f"https://api.pi42.com/v1/market/ticker24Hr/{contract_pair}"

    try:
        # Send the GET request to the API
        response = requests.get(full_url)
        response.raise_for_status()  # Raise an error for HTTP 4xx/5xx responses

        # Parse the JSON response data
        response_data = response.json()
        print(response_data)
        return response_data
    
    except requests.exceptions.HTTPError as err:
        # Handle HTTP errors specifically
        print(f"Error: {err.response.text if err.response else err}")
    except Exception as e:
        # Handle any other unexpected errors
        print(f"An unexpected error occurred: {str(e)}")

In [5]:
def get_24hr_ticker_update(contract_pair):

    # Validate the input
    if not contract_pair:
        print("Invalid contract pair. Please enter a valid contract pair (e.g., btc, eth).")
        return

    # Construct the full URL for the API request using the provided contract pair
    full_url = f"https://api.pi42.com/v1/market/ticker24Hr/{contract_pair}"

    try:
        # Send the GET request to the API
        response = requests.get(full_url)
        response.raise_for_status()  # Raise an error for HTTP 4xx/5xx responses

        # Parse the JSON response data
        response_data = response.json()

        return response_data
    
    except requests.exceptions.HTTPError as err:
        # Handle HTTP errors specifically
        print(f"Error: {err.response.text if err.response else err}")
    except Exception as e:
        # Handle any other unexpected errors
        print(f"An unexpected error occurred: {str(e)}")

def get_current_price(ticker):
    return float(get_24hr_ticker_update(ticker)['data']["c"])

In [6]:
upper_pct = 0.001
lower_pct = 0.001
qty = 0.002
symbol = 'BTCUSDT'

current_price = get_current_price(symbol)
upper_limit_price = int(current_price * (1 + upper_pct))
lower_limit_price = int(current_price * (1 - lower_pct))

place_order('BTCUSDT', upper_limit_price, 'LIMIT', qty, 'BUY', False)

API KEY:  b082b62c54596daa2bb71ff276894a12
API SECRET:  4c3d89a7bfb1f0223a72ee7bcad34202
https://fapi.pi42.com/v1/order/place-order
Payload being sent: {
    "timestamp": "1731932491372",
    "placeType": "ORDER_FORM",
    "quantity": 0.002,
    "side": "BUY",
    "symbol": "BTCUSDT",
    "type": "LIMIT",
    "reduceOnly": false,
    "marginAsset": "INR",
    "deviceType": "WEB",
    "userCategory": "EXTERNAL",
    "price": 90571,
    "stopPrice": 90571
}
Headers being sent: {'api-key': 'b082b62c54596daa2bb71ff276894a12', 'signature': 'cc9edba255eb08cbe3164fec76b952308625d5968b2bf80c5bb30f95dfef1d5e'}
Error: 400 Client Error: Bad Request for url: https://fapi.pi42.com/v1/order/place-order


In [7]:
def custom_round_to_10_not_5(number):
    rem = number % 10

    if rem == 5:
        return number
    if rem < 5:
        number -= rem
        return number
    if rem > 5:
        number += (10- rem)
        return number


for i in range(90000, 100000):
    print("Limit Order for", i, "Will get Placed at -->", custom_round_to_10_not_5(i))

Limit Order for 90000 Will get Placed at --> 90000
Limit Order for 90001 Will get Placed at --> 90000
Limit Order for 90002 Will get Placed at --> 90000
Limit Order for 90003 Will get Placed at --> 90000
Limit Order for 90004 Will get Placed at --> 90000
Limit Order for 90005 Will get Placed at --> 90005
Limit Order for 90006 Will get Placed at --> 90010
Limit Order for 90007 Will get Placed at --> 90010
Limit Order for 90008 Will get Placed at --> 90010
Limit Order for 90009 Will get Placed at --> 90010
Limit Order for 90010 Will get Placed at --> 90010
Limit Order for 90011 Will get Placed at --> 90010
Limit Order for 90012 Will get Placed at --> 90010
Limit Order for 90013 Will get Placed at --> 90010
Limit Order for 90014 Will get Placed at --> 90010
Limit Order for 90015 Will get Placed at --> 90015
Limit Order for 90016 Will get Placed at --> 90020
Limit Order for 90017 Will get Placed at --> 90020
Limit Order for 90018 Will get Placed at --> 90020
Limit Order for 90019 Will get 